In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms, datasets
import PIL.Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import glob
from torchvision.datasets import ImageFolder
import matplotlib.pyplot as plt

In [3]:
transform = transforms.Compose([transforms.Resize(255),
                                 transforms.CenterCrop(224),
                                 transforms.ToTensor()])

path = "../week6/data/alexdata/train"
test = "../week6/data/alexdata/validation"
device = torch.device("cuda")

train=ImageFolder(path, transform = transform)
test = ImageFolder(test, transform = transform)

train_loader = DataLoader(train, batch_size = 100)
test_loader = DataLoader(train, batch_size = 100)

In [23]:


# Assuming you already have your dataset loaded, let's define a simple neural network
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.fc1 = nn.Linear(12*12*64, 128)
        self.fc2 = nn.Linear(128, 2)  # 2 classes: cat and dog

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2, 2)
        x = x.view(-1, 12*12*64)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        x = torch.relu(x)
        return x

# Load your dataset here
# Assuming you have train_loader and test_loader

# Instantiate the model
model = NeuralNetwork()

# Define your loss function and optimizer with L2 regularization
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.001)  # L2 regularization with weight decay

# Training loop
epochs = 10
for epoch in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        if i % 10 == 9:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')

# Test the model
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
       
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))

# To observe the impact of regularization on the weight parameters, you can access them using model.parameters()


ValueError: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([2025, 2])) is deprecated. Please ensure they have the same size.

In [24]:
outputs.shape

torch.Size([2025, 2])

In [25]:
labels.shape

torch.Size([100])